# NLP Embeddings II

# PreComputed Embeddings: 

Waht if we didnt have to comput the emebddings ourselves?

We are going to use a pre-trained word embedding for finding word analogies and equivalence. This exercise can be used as an Intrinsic Evaluation for the word embedding performance. In this notebook, you will apply linear algebra operations using NumPy to find analogies between words manually. This will help you to prepare for this week's assignment.

In [ ]:
import pandas as pd # Library for Dataframes 
import numpy as np # Library for math functions
import pickle # Python object serialization library. Not secure

word_embeddings = pickle.load( open( "word_embeddings_subset.p", "rb" ) )
len(word_embeddings) # there should be 243 words that will be used in this assignment

Now that the model is loaded, we can take a look at the word representations. First, note that the _word_embeddings_ is a dictionary. Each word is the key to the entry, and the value is its corresponding vector presentation. Remember that square brackets allow access to any entry if the key exists. 

In [ ]:
countryVector = word_embeddings['country'] # Get the vector representation for the word 'country'
print(type(countryVector)) # Print the type of the vector. Note it is a numpy array
print(countryVector) # Print the values of the vector.  

It is important to note that we store each vector as a NumPy array. It allows us to use the linear algebra operations on it. 

The vectors have a size of 300, while the vocabulary size of Google News is around 3 million words! 

In [ ]:
#Get the vector for a given word:
def vec(w):
    return word_embeddings[w]

## Operating on word embeddings

Remember that understanding the data is one of the most critical steps in Data Science. Word embeddings are the result of machine learning processes and will be part of the input for further processes. These word embedding needs to be validated or at least understood because the performance of the derived model will strongly depend on its quality.

Word embeddings are multidimensional arrays, usually with hundreds of attributes that pose a challenge for its interpretation. 

In this notebook, we will visually inspect the word embedding of some words using a pair of attributes. Raw attributes are not the best option for the creation of such charts but will allow us to illustrate the mechanical part in Python. 

In the next cell, we make a beautiful plot for the word embeddings of some words. Even if plotting the dots gives an idea of the words, the arrow representations help to visualize the vector's alignment as well.

In [ ]:
import matplotlib.pyplot as plt # Import matplotlib

words = ['oil', 'gas', 'happy', 'sad', 'city', 'town', 'village', 'country', 'continent', 'petroleum', 'joyful']

bag2d = np.array([vec(word) for word in words]) # Convert each word to its vector representation

fig, ax = plt.subplots(figsize = (10, 10)) # Create custom size image

col1 = 3 # Select the column for the x axis
col2 = 2 # Select the column for the y axis

# Print an arrow for each word
for word in bag2d:
    ax.arrow(0, 0, word[col1], word[col2], head_width=0.005, head_length=0.005, fc='r', ec='r', width = 1e-5)

    
ax.scatter(bag2d[:, col1], bag2d[:, col2]); # Plot a dot for each word

# Add the word label over each dot in the scatter plot
for i in range(0, len(words)):
    ax.annotate(words[i], (bag2d[i, col1], bag2d[i, col2]))


plt.show()

Note that similar words like 'village' and 'town' or 'petroleum', 'oil', and 'gas' tend to point in the same direction. Also, note that 'sad' and 'happy' looks close to each other; however, the vectors point in opposite directions.

In this chart, one can figure out the angles and distances between the words. Some words are close in both kinds of distance metrics.

## Word distance

Now plot the words 'sad', 'happy', 'town', and 'village'. In this same chart, display the vector from 'village' to 'town' and the vector from 'sad' to 'happy'. Let us use NumPy for these linear algebra operations.

In [ ]:
words = ['sad', 'happy', 'town', 'village']

bag2d = np.array([vec(word) for word in words]) # Convert each word to its vector representation

fig, ax = plt.subplots(figsize = (10, 10)) # Create custom size image

col1 = 3 # Select the column for the x axe
col2 = 2 # Select the column for the y axe

# Print an arrow for each word
for word in bag2d:
    ax.arrow(0, 0, word[col1], word[col2], head_width=0.0005, head_length=0.0005, fc='r', ec='r', width = 1e-5)
    
# print the vector difference between village and town
village = vec('village')
town = vec('town')
diff = town - village
ax.arrow(village[col1], village[col2], diff[col1], diff[col2], fc='b', ec='b', width = 1e-5)

# print the vector difference between village and town
sad = vec('sad')
happy = vec('happy')
diff = happy - sad
ax.arrow(sad[col1], sad[col2], diff[col1], diff[col2], fc='b', ec='b', width = 1e-5)


ax.scatter(bag2d[:, col1], bag2d[:, col2]); # Plot a dot for each word

# Add the word label over each dot in the scatter plot
for i in range(0, len(words)):
    ax.annotate(words[i], (bag2d[i, col1], bag2d[i, col2]))


plt.show()


## Linear algebra on word embeddings

In the lectures, we saw the analogies between words using algebra on word embeddings. Let us see how to do it in Python with Numpy.

To start, get the **norm** of a word in the word embedding.

In [ ]:
print(np.linalg.norm(vec('town'))) # Print the norm of the word town
print(np.linalg.norm(vec('sad'))) # Print the norm of the word sad

## Predicting capitals

Now, applying vector difference and addition, one can create a vector representation for a new word. For example, we can say that the vector difference between 'France' and 'Paris' represents the concept of Capital.

One can move from the city of Madrid in the direction of the concept of Capital, and obtain something close to the corresponding country to which Madrid is the Capital.

In [ ]:
capital = vec('France') - vec('Paris')
country = vec('Madrid') + capital

print(country[0:5]) # Print the first 5 values of the vector

We can observe that the vector 'country' that we expected to be the same as the vector for Spain is not exactly it.

In [ ]:
diff = country - vec('Spain')
print(diff[0:10])

So, we have to look for the closest words in the embedding that matches the candidate country. If the word embedding works as expected, the most similar word must be 'Spain'. Let us define a function that helps us to do it. We will store our word embedding as a DataFrame, which facilitate the lookup operations based on the numerical vectors.

In [ ]:
# Create a dataframe out of the dictionary embedding. This facilitate the algebraic operations
keys = word_embeddings.keys()
data = []
for key in keys:
    data.append(word_embeddings[key])

embedding = pd.DataFrame(data=data, index=keys)
# Define a function to find the closest word to a vector:
def find_closest_word(v, k = 1):
    # Calculate the vector difference from each word to the input vector
    diff = embedding.values - v 
    # Get the norm of each difference vector. 
    # It means the squared euclidean distance from each word to the input vector
    delta = np.sum(diff * diff, axis=1)
    # Find the index of the minimun distance in the array
    i = np.argmin(delta)
    # Return the row name for this item
    return embedding.iloc[i].name


In [ ]:
# Print some rows of the embedding as a Dataframe
embedding.head(10)

Now let us find the name that corresponds to our numerical country:

In [ ]:
find_closest_word(country)

## Predicting other Countries

In [ ]:
find_closest_word(vec('Italy') - vec('Rome') + vec('Madrid'))

In [ ]:
print(find_closest_word(vec('Berlin') + capital))
print(find_closest_word(vec('Beijing') + capital))

However, it does not always work.

In [ ]:
print(find_closest_word(vec('Lisbon') + capital))

## Represent a sentence as a vector

A whole sentence can be represented as a vector by summing all the word vectors that conform to the sentence. Let us see. 

In [ ]:
doc = "Spain petroleum city king"
vdoc = [vec(x) for x in doc.split(" ")]
doc2vec = np.sum(vdoc, axis = 0)
doc2vec

In [ ]:
find_closest_word(doc2vec)

## Embedding Models

What if we could donwload a model with already all the English dictionary in it?

In [ ]:
#$%pip install gensim
%pip install scipy==1.12

from scipy.linalg import triu

import gensim
import gensim.downloader as api

In [ ]:
# the model I'd like to use
#model_nope = api.load("word2vec-google-news-300")

# too large, let's try something more modest
model = api.load("glove-wiki-gigaword-100")

# each word has an encoding
model['boat']

In [ ]:
#and you can search for "similar" words (words that could occupy the same space)
model.most_similar('boat')

In [ ]:
model.most_similar('book')

# Now for the jaw dropping part

In [ ]:
# you can "add" or subtract tendencies
model.most_similar(positive=['woman','king']
                    #,negative=['man']
                    ,topn=1)

model.most_similar(positive=['woman','king'],negative=['man'],topn=1)

In [ ]:
model.most_similar(positive=['woman','boy'],negative=['man'],topn=1)

In [ ]:
#if at first you don't succeed
model.most_similar(positive=['grape','beer'],negative=['barley'],topn=1)

In [ ]:
model.most_similar(positive=['lisbon','spain'],negative=['madrid'],topn=1)

## Cosine similarity

![cosine](https://www.oreilly.com/api/v2/epubs/9781788295758/files/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png)

In [ ]:
#why does this work?
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([model['king']],[model['duck']])

In [ ]:
cosine_similarity([model['king']],[model['man']])

In [ ]:
cosine_similarity([model['king']],[model['crown']])

In [ ]:
cosine_similarity([model['king']],[model['monarch']])

In [ ]:
#now that we established a baseline for similarity...

In [ ]:
cosine_similarity([model['king']-model['queen']],[model['son']-model['daughter']])

In [ ]:
cosine_similarity([model['russia']-model['moscow']],[model['poland']-model['warsaw']])

## Embedding Models and APIs
(A tease for what a RAG system will be one day)



This code demonstrates how to utilize embeddings to search and retrieve specific content from a document using a vector database. The document, in this case, a PDF file, is first loaded and split into manageable chunks using a text splitter. These chunks are then embedded, converting the text into high-dimensional vectors that capture semantic meaning. The embeddings are stored in a vector database, such as Chroma, allowing for efficient similarity searches. By inputting a query like "Can I onboard data using SQL in PowerBI?", the system retrieves the most relevant document chunks, enabling precise and efficient document-based search functionality.








### Document and Documentation Search


In [ ]:
#!pip install langchain_openai
#!pip install langchain_chroma
#!pip install langchain_chroma

In [1]:
## OPEN AI EMBEDDINGS:
from langchain_openai import OpenAIEmbeddings
import os
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# Get your API KEY here: https://platform.openai.com/api-keys
# or
# migrate the embeddings to a model from HuggingFace
API_KEY = '----'

# Create the embeddings function
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key = API_KEY)

# create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
print('Cell finished')

Cell finished


In [3]:
# load the document and split it into chunks
document_dir = "./"
filename = "powerbi_book.pdf"
file_path = os.path.join(document_dir, filename)

pages = PyPDFLoader(file_path).load_and_split()
docs = text_splitter.split_documents(pages)

# load it into Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
print('Cell finished')

Cell finished


In [4]:
# query it

#user_question = 'How do I build one of those charts that look like a Swiss Cheese?'
user_question = 'Can I onboard data using SQL in powerBi?'
#user_question = 'Tell me about Line Charts

#user_question = "How can I do a pie chart in PowerBI?"
docs = db.similarity_search(user_question, k=10)

# print results
print(docs[0].page_content)

137 C H A P T E R  4  |  Using Power BI Desktop 
 Figure 4-3: Before importing data from an SQL 
database, you need to choose the loading method. 
Let’s take a moment to learn about this 
connection option because it is an important 
one and will help shed more light on how Power 
BI connections work. 
When you choose Import, Power BI Desktop 
connects to the database, loads the information, 
and stores it within its internal data model. You 
can then work on your data in Power BI Desktop 
without being connected to the database. You 
will only need a connection when you want to 
refresh the data. 
With DirectQuery, Power BI Desktop does not 
load the data into its internal database. Instead, 
it runs a query to the original database every 
time it needs to draw a chart or, in general, run a 
query. Thus, the connection between Power BI 
Desktop and the database will be permanent. 
The contrast in the query timings reflects a key 
difference: when you use Import, you are 
working with 

In [5]:
def _get_document_prompt(docs):
    prompt = '\n'
    for doc in docs:
        prompt += '\nContent:\n'
        prompt += doc.page_content + '\n\n'
    return prompt

print(_get_document_prompt(docs))



Content:
137 C H A P T E R  4  |  Using Power BI Desktop 
 Figure 4-3: Before importing data from an SQL 
database, you need to choose the loading method. 
Let’s take a moment to learn about this 
connection option because it is an important 
one and will help shed more light on how Power 
BI connections work. 
When you choose Import, Power BI Desktop 
connects to the database, loads the information, 
and stores it within its internal data model. You 
can then work on your data in Power BI Desktop 
without being connected to the database. You 
will only need a connection when you want to 
refresh the data. 
With DirectQuery, Power BI Desktop does not 
load the data into its internal database. Instead, 
it runs a query to the original database every 
time it needs to draw a chart or, in general, run a 
query. Thus, the connection between Power BI 
Desktop and the database will be permanent. 
The contrast in the query timings reflects a key 
difference: when you use Import, you are 
wo